In [13]:
import fasttext
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import re

In [61]:
df = pd.read_csv('questions3.csv', delimiter=';', encoding='utf-8')
df.head()

,question,response,Unnamed: 2
0,Какая квалификация присваивается выпускникам т...,"Выпускникам специальности 09.02.01 ""Компьютерн...",NaN
1,Какая квалификация присваивается выпускникам т...,"Выпускникам специальности 09.02.06 ""Сетевое и ...",NaN
2,Какая квалификация присваивается выпускникам с...,Выпускникам специальности 09.02.08 присваивает...,NaN
3,Какая квалификация присваивается выпускникам с...,Выпускникам специальности 10.02.05 присваивает...,NaN
4,Какая квалификация присваивается выпускникам с...,Выпускникам специальности 25.02.08 присваивает...,NaN


In [63]:
def clean_text(string: str):
    text = string.replace(".", '').replace("?", ' ').replace('-', ' ').replace('"', '').rstrip()
    return text.lower()

In [64]:
questions = [clean_text(i) for i in df.question]
questions

['какая квалификация присваивается выпускникам техникума по специальности 090201 компьютерные системы и комплексы',
 'какая квалификация присваивается выпускникам техникума по специальности 090206 сетевое и системное администрирование',
 'какая квалификация присваивается выпускникам специальности 090208 web разработка',
 'какая квалификация присваивается выпускникам специальности 100205 обеспечение информационной безопасности автоматизированных систем',
 'какая квалификация присваивается выпускникам специальности 250208 эксплуатация беспилотных авиационных систем',
 'какая квалификация присваивается выпускникам специальности 400204 юриспруденция',
 'какая квалификация присваивается выпускникам специальности 090207 информационные системы и программирование',
 'как осуществляется оплата обучения — сразу за учебный год',
 'когда начинается прием документов в техникум',
 'когда начинается учебный год у студентов',
 'по какому адресу учатся студенты',
 'можно ли оплачивать обучение из средс

In [80]:
class_responses = {}
responses_class = {}
array_classifications = []

index_class = 1
for resp in df.response:
    
    if resp not in class_responses.values():
        class_responses[index_class] = resp
        responses_class[resp] = index_class
        array_classifications.append(index_class)
        index_class += 1
    
    else:
        array_classifications.append(responses_class[resp])

{'какая квалификация присваивается выпускникам техникума по специальности 090201 компьютерные системы и комплексы': 1, 'какая квалификация присваивается выпускникам техникума по специальности 090206 сетевое и системное администрирование': 2, 'какая квалификация присваивается выпускникам специальности 090208 web разработка': 3, 'какая квалификация присваивается выпускникам специальности 100205 обеспечение информационной безопасности автоматизированных систем': 4, 'какая квалификация присваивается выпускникам специальности 250208 эксплуатация беспилотных авиационных систем': 5, 'какая квалификация присваивается выпускникам специальности 400204 юриспруденция': 6, 'какая квалификация присваивается выпускникам специальности 090207 информационные системы и программирование': 7, 'как осуществляется оплата обучения — сразу за учебный год': 8, 'когда начинается прием документов в техникум': 9, 'когда начинается учебный год у студентов': 10, 'по какому адресу учатся студенты': 11, 'можно ли опла

In [84]:
x_train, x_test, y_train, y_test = train_test_split(questions, array_classifications, test_size = 0.2)

['какая квалификация 090206',
 'выплачивается ли студентам бюджетникам стипендия',
 'какая квалификация присваивается выпускникам специальности 090207 информационные системы и программирование',
 'кто получает квалификацию по специальности 090206 сетевое и системное администрирование',
 'можно ли использовать средства материнского капитала для оплаты обучения',
 'разрешено ли отправлять документы в приемную комиссию в электронном формате',
 'есть ли стипендия для бюджетников',
 'кто получает квалификацию по специальности 100205 обеспечение информационной безопасности автоматизированных систем',
 'какая квалификация присваивается выпускникам техникума по специальности 090206 сетевое и системное администрирование',
 'кто получает квалификацию по специальности 090207 информационные системы и программирование',
 'можно ли оплачивать обучение из средств материнского капитала',
 'кто получает квалификацию по специальности 090201 компьютерные системы и комплексы',
 'какая квалификация присваи

In [87]:
with open('train.txt', 'w', encoding='utf-8') as f:
    for text, label in zip(x_train, y_train):
        f.writelines(f'__label__{label} {text}\n')

with open('test.txt', 'w', encoding='utf-8') as f:
    for text, label in zip(x_test, y_test):
        f.writelines(f'__label__{label} {text}\n')
        

In [88]:
model = fasttext.train_supervised('train.txt', lr=0.5, epoch=25)

In [89]:
def print_results(sample_size, precision, recall):
    precision   = round(precision, 2)
    recall      = round(recall, 2)
    print(f'{sample_size=}')
    print(f'{precision=}')
    print(f'{recall=}')

In [90]:
print_results(*model.test('test.txt'))

sample_size=9
precision=0.22
recall=0.22


In [123]:

what_predict = clean_text("Как происходит оплата")
pred = model.predict(what_predict)[0][0].split("__")

print(class_responses)
print(pred)
print(class_responses[int(pred[2])])

{1: 'Выпускникам специальности 09.02.01 "Компьютерные системы и комплексы" присваивается квалификация "Техник по компьютерным системам"', 2: 'Выпускникам специальности 09.02.06 "Сетевое и системное администрирование" присваивается квалификация "Сетевой и системный администратор"', 3: 'Выпускникам специальности 09.02.08 присваивается квалификация "Разработчик веб-приложений"', 4: 'Выпускникам специальности 10.02.05 присваивается квалификация "Техник по защите информации"', 5: 'Выпускникам специальности 25.02.08 присваивается квалификация "Оператор беспилотных летательных аппаратов"', 6: 'Выпускникам специальности 40.02.04 присваивается квалификация "Юрист"', 7: 'Выпускникам специальности 09.02.07 присваивается квалификация в зависимости от выбранного направления: 1. Администратор баз данных 2. Специалист по информационным системам 3. Разработчик WEB и мультимедийных приложений 4. Программист 5. Специалист по тестированию в области информационных технологий', 8: 'Оплата обучения осуществ